To fix list:
- The 'year' scraping part still doesn't work

For future reference:
2001 to 2006 has format:
columns=["Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "FF", "FA"]

2007 to 2009 has format:
columns=["Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "GA", "I50", "FF", "FA", "AF", "SC"]

2010 to 2022 has format:
columns=["Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "GA", "I50", "CL", "CG", "R50", "FF", "FA", "AF", "SC"]

Note that 2010 onwards also has advanced stats available: add afl/footy/ft_match_statistics?mid=5089&advv=Y

In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from tqdm import tqdm

try:
    with open('invalid_match_ids.txt') as f:
        match_ids = f.read().splitlines()
    print(match_ids)
    first_game = int(match_ids[0])
    last_game = int(match_ids[5])
except:
    first_game = 1
    last_game = 10741
    match_id = range(first_game, last_game+1)

dfs = []
bad_match_ids = []
for match_id in match_ids[0:5]:
    match_id = int(match_id)
    try:
        print(match_id)
        url = f"https://www.footywire.com/afl/footy/ft_match_statistics?mid={match_id}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        # Find the table element with the match statistics
        match_stats_table = soup.find("table", {"id": "match-statistics-div"})
        if match_stats_table is None:
            print(f"No match statistics div found for match ID {match_id}")
            continue

        # Find the round number and year
        if match_id < 1840:
            year_round = soup.find('td', {"class": "lnorm", "height":"22"})
            round = year_round.text[0:year_round.text.find(',')]
            year = year_round.text[-4:]
        else:
            year_round = soup.find('td', {"class": "lnorm", "height":"22"})
            round = year_round.text[0:year_round.text.find(',')]
            year_round = soup.find('td', {"class": "lnorm", "height":"22"}).find_next('td', {"class": "lnorm", "height":"22"})
            year = year_round.text[-4:]

        if 'final' in round.lower():
            print(f"There is 'final' in this round: {round}")
            continue

        # Find the rows with player statistics
        player_stats_rows = match_stats_table.find_all("tr", {"class": ["darkcolor", "lightcolor"], "onmouseover": "this.className='highlightcolor';", "onmouseout": ["this.className='darkcolor';", "this.className='lightcolor';"]})
        first_team = soup.find("td", {"class": "innertbtitle", "align": "left"}).find("b").text
        first_team = first_team[:first_team.find("Match Statistics")].strip()
        second_team = soup.find("td", {"class": "innertbtitle", "align": "left"}).find_next("td", {"class": "innertbtitle", "align": "left"}).find("b").text
        second_team = second_team[:second_team.find("Match Statistics")].strip()

        if 'defeats' in soup.title.text.lower():
            winner = first_team
        else:
            winner = second_team

        # Extract the data from each row and create a pandas dataframe
        data = []
        for i, row in enumerate(player_stats_rows):
            team = first_team if i < len(player_stats_rows)/2 else second_team
            cols = row.find_all("td")
            cols = [col.text.strip() for col in cols]
            if len(cols) == 11:
                result = 1 if team == winner else 0
                new_row = [year, round, team, result] + cols
                data.append(new_row)
                columns=["Year", "Round", "Team Name", "Win", "Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "FF", "FA"]
            elif len(cols) == 15:
                result = 1 if team == winner else 0
                new_row = [year, round, team, result] + cols
                data.append(new_row)
                columns=["Year", "Round", "Team Name", "Win", "Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "GA", "I50", "FF", "FA", "AF", "SC"]
            elif len(cols) == 18:
                result = 1 if team == winner else 0
                new_row = [year, round, team, result] + cols
                data.append(new_row)
                columns=["Year", "Round", "Team Name", "Win", "Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "GA", "I50", "CL", "CG", "R50", "FF", "FA", "AF", "SC"]

        if data:
            df = pd.DataFrame(data, columns=columns)
            df["Match ID"] = match_id   

            # Get Brownlow votes
            brownlow_votes = soup.find(string=re.compile("Brownlow Votes:"))
            print("Votes:{}".format(brownlow_votes))
            if brownlow_votes:
                votes = {}
                for i in range(3):
                    player = brownlow_votes.find_next("a").text.strip()
                    player_parts = player.split(" ")
                    last_name = player_parts[-1]
                    first_initial = player_parts[0][0]
                    player_reversed = " ".join(player_parts[::-1])
                    votes[player_reversed] = 3-i
                    brownlow_votes = brownlow_votes.find_next("a")
                print(votes)
                df["Votes"] = df["Player"].apply(lambda x: votes.get(x.split()[-1] + ' ' + x.split()[0][0], 0))
            else:
                df["Votes"] = 0

            dfs.append(df)
            print(f"Processed match ID {match_id}")
    except Exception as e:
        print(f"Error processing match ID {match_id}: {e}")
        bad_match_ids.append(match_id)

# Concatenate all dataframes into a single one
df_final = pd.concat(dfs)

# Save the data to a CSV file
# df_final.to_csv("player_stats.csv", index=False)


['22', '31', '58', '210', '233', '306', '383', '405', '493', '585', '600', '616', '619', '621', '625', '636', '641', '697', '826', '845', '946', '958', '978', '1010', '1054', '1062', '1102', '1264', '1305', '1307', '1312', '1358', '1443', '1470', '1484', '1525', '1568', '1644', '1809', '1822', '1848', '1849', '1863', '1867', '1890', '1918', '1925', '1972', '2012', '2043', '2056', '2063', '2071', '2147', '2162', '2169', '2197', '2405', '2420', '2430', '2468', '2513', '2539', '2560', '2951', '3006', '3016', '3034', '3040', '3065', '3490', '3534', '3570', '3614', '3619', '3623', '3748', '3754', '3805', '3847', '3876', '3885', '4158', '4160', '4167', '4195', '4243', '4255', '4278', '4309', '4310', '4320', '4379', '4435', '4481', '4493', '4571', '4675', '4705', '4733', '4738', '4847', '4855', '4874', '4900', '4933', '4971', '4973', '4987', '4988', '5045', '5048', '5062', '5072', '5084', '5130', '5148', '5184', '5202', '5219', '5228', '5244', '5263', '5269', '5274', '5276', '5279', '5290', '

In [2]:
import pandas as pd

with open('player_stats.csv', 'r') as f:
    lines = f.readlines()
    lines = [line.replace('\'', '') for line in lines]

# sum votes, group by match_id
df = pd.read_csv('player_stats.csv')
df = df.groupby(['Match ID']).sum().reset_index()
df = df[['Match ID', 'Votes']]
# get all match_ids with votes != 6
df = df[df['Votes'] != 6]
df['Match ID'].reset_index()

,index,Match ID
0,21,22
1,30,31
2,57,58
3,200,210
4,223,233
...,...,...
243,6358,10704
244,6360,10706
245,6368,10714
246,6378,10724


In [ ]:
# Write match ids to file
with open('invalid_match_ids.txt', 'w') as f:
    for match_id in df['Match ID']:
        f.write(f'{match_id}\n')